In [1]:
# Not all of these are currently used, some here for future use
import pandas as pd
import numpy as np
import sportsdataverse 
import matplotlib.pyplot as plt
import bs4
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from PIL import Image
import urllib
import requests
from datetime import date
from io import StringIO
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Rectangle
import warnings
from kenpompy import misc
# Filter out FutureWarnings
warnings.filterwarnings('ignore', category=FutureWarning)

import imageio.v2 as imageio
from io import BytesIO
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

In [2]:
import kenpompy

In [3]:
# Supposed to be used for future, will add logos and a user interface
mbb_df = sportsdataverse.mbb.load_mbb_team_boxscore(seasons=range(2023,2024))
mbb_df.head()

100%|██████████| 1/1 [00:00<00:00,  1.19it/s]


,game_id,season,season_type,game_date,game_date_time,team_id,team_uid,team_slug,team_location,team_name,team_abbreviation,team_display_name,team_short_display_name,team_color,team_alternate_color,team_logo,team_home_away,team_score,team_winner,assists,blocks,defensive_rebounds,field_goal_pct,field_goals_made,field_goals_attempted,flagrant_fouls,fouls,free_throw_pct,free_throws_made,free_throws_attempted,largest_lead,offensive_rebounds,steals,team_turnovers,technical_fouls,three_point_field_goal_pct,three_point_field_goals_made,three_point_field_goals_attempted,total_rebounds,total_technical_fouls,total_turnovers,turnovers,opponent_team_id,opponent_team_uid,opponent_team_slug,opponent_team_location,opponent_team_name,opponent_team_abbreviation,opponent_team_display_name,opponent_team_short_display_name,opponent_team_color,opponent_team_alternate_color,opponent_team_logo,opponent_team_score
0,401522202,2023,3,2023-04-03,2023-04-03 21:20:00-04:00,21,s:40~l:41~t:21,san-diego-state-aztecs,San Diego State,Aztecs,SDSU,San Diego State Aztecs,San Diego St,BF2C37,a8996e,https://a.espncdn.com/i/teamlogos/ncaa/500/21.png,away,59,False,7,2,22,32.2,19,59,0,20,75.0,15,20,4,12,7,0,0,26.1,6,23,34,0,12,12,41,s:40~l:41~t:41,uconn-huskies,UConn,Huskies,CONN,UConn Huskies,UConn,0c2340,f1f2f3,https://a.espncdn.com/i/teamlogos/ncaa/500/41.png,76
1,401522202,2023,3,2023-04-03,2023-04-03 21:20:00-04:00,41,s:40~l:41~t:41,uconn-huskies,UConn,Huskies,CONN,UConn Huskies,UConn,0c2340,f1f2f3,https://a.espncdn.com/i/teamlogos/ncaa/500/41.png,home,76,True,13,6,30,43.4,23,53,0,15,88.9,24,27,17,10,8,0,0,35.3,6,17,40,0,13,13,21,s:40~l:41~t:21,san-diego-state-aztecs,San Diego State,Aztecs,SDSU,San Diego State Aztecs,San Diego St,BF2C37,a8996e,https://a.espncdn.com/i/teamlogos/ncaa/500/21.png,59
2,401522201,2023,3,2023-04-01,2023-04-01 21:16:00-04:00,2390,s:40~l:41~t:2390,miami-hurricanes,Miami,Hurricanes,MIA,Miami Hurricanes,Miami,004325,f0f0f0,https://a.espncdn.com/i/teamlogos/ncaa/500/239...,away,59,False,10,1,18,32.3,20,62,0,12,100.0,12,12,0,14,8,0,0,35.0,7,20,32,0,9,9,41,s:40~l:41~t:41,uconn-huskies,UConn,Huskies,CONN,UConn Huskies,UConn,0c2340,f1f2f3,https://a.espncdn.com/i/teamlogos/ncaa/500/41.png,72
3,401522201,2023,3,2023-04-01,2023-04-01 21:16:00-04:00,41,s:40~l:41~t:41,uconn-huskies,UConn,Huskies,CONN,UConn Huskies,UConn,0c2340,f1f2f3,https://a.espncdn.com/i/teamlogos/ncaa/500/41.png,home,72,True,19,5,28,49.1,28,57,0,11,53.8,7,13,20,13,5,1,0,34.6,9,26,41,0,15,15,2390,s:40~l:41~t:2390,miami-hurricanes,Miami,Hurricanes,MIA,Miami Hurricanes,Miami,004325,f0f0f0,https://a.espncdn.com/i/teamlogos/ncaa/500/239...,59
4,401522200,2023,3,2023-04-01,2023-04-01 18:09:00-04:00,2226,s:40~l:41~t:2226,florida-atlantic-owls,Florida Atlantic,Owls,FAU,Florida Atlantic Owls,FAU,004B85,bb2f4c,https://a.espncdn.com/i/teamlogos/ncaa/500/222...,away,71,False,6,2,26,44.2,23,52,0,17,76.2,16,21,14,8,6,1,0,40.9,9,22,34,0,10,10,21,s:40~l:41~t:21,san-diego-state-aztecs,San Diego State,Aztecs,SDSU,San Diego State Aztecs,San Diego St,BF2C37,a8996e,https://a.espncdn.com/i/teamlogos/ncaa/500/21.png,72


In [4]:
from kenpompy.utils import login

In [5]:
browser = login(username, password)

In [6]:
#Built Atop the KenpomPy Template
def get_gameplan(team, browser, season=None):
    """
    Scrapes the teams (https://kenpom.com) into a list.
    Args:
        team (str, required): Used to determine which team to get the gameplan df for 
        browser (mechanicalsoul StatefulBrowser): Authenticated browser with full access to kenpom.com generated
            by the `login` function
        season (str, optional): Used to define different seasons. 2002 is the earliest available season.
    Returns:
        team_list (list): List containing all valid teams for the given season on kenpom.com.
    """

    url = "https://kenpom.com"
    if season == None:
        url = url + '/gameplan.php?team=' + team
    else:
        url = url + '/gameplan.php?team=' + team + str(season)
    if team[-3:] == 'A&M':
        shortened = team[:-4]
        url = 'https://kenpom.com' + '/gameplan.php?team=' + shortened + '+A%26M'    
    if team == 'Texas A&M Corpus Chris':
        url = 'https://kenpom.com/gameplan.php?team=Texas%20A%26M%20Corpus%20Chris'
    if ' ' in url:
        url = url.replace(' ', '+')

    print(url)

    browser.open(url)
    gameplan = browser.get_current_page()
    table = gameplan.find_all('table')[0]
    gameplan_df = pd.read_html(str(table))
    gameplan_df = gameplan_df[0]
    
    gameplan_df.columns = ['date', 'opponent_kp_rank', 'opponent', 'result', 'home_or_away', 'pace', 'off_eff', 'off_eff_rank_tm', 'off_eFG%', 'off_to%',
                           'off_reb%', 'off_ftr', 'off_2p%', 'off_2p%_rank_tm', 'off_3p_shooting', 'off_3p%', 'off_3pa%', 'def_eff', 'def_eff_rank_tm', 
                           'def_eFG%', 'def_to%', 'def_reb%', 'def_ftr', 'def_2p%', 'def_2p%_rank_tm', 'def_3p_shooting', 'def_3p%_', 'def_3pa%']
    gameplan_df.drop(gameplan_df.tail(5).index, inplace = True)
    gameplan_df['pace'] = gameplan_df['pace'].astype(int)
    gameplan_df['off_eff'] = gameplan_df['off_eff'].astype(float)
    gameplan_df['def_eff'] = gameplan_df['def_eff'].astype(float)


    
    return gameplan_df

In [7]:
team1_df = get_gameplan('Alabama', browser)

https://kenpom.com/gameplan.php?team=Alabama


In [8]:
team1_df

,date,opponent_kp_rank,opponent,result,home_or_away,pace,off_eff,off_eff_rank_tm,off_eFG%,off_to%,off_reb%,off_ftr,off_2p%,off_2p%_rank_tm,off_3p_shooting,off_3p%,off_3pa%,def_eff,def_eff_rank_tm,def_eFG%,def_to%,def_reb%,def_ftr,def_2p%,def_2p%_rank_tm,def_3p_shooting,def_3p%_,def_3pa%
0,Mon Nov 6,110,Morehead St.,"W, 105-73",H,74,142.8,1,64.9,17.7,46.7,66.7,22-34,64.7,10-23,43.5,40.4,99.3,4,49.2,20.4,33.3,30.6,20-39,51.3,7-23,30.4,37.1
1,Fri Nov 10,38,Indiana St.,"W, 102-80",H,74,137.7,1,61.7,14.8,40.0,42.2,20-41,48.8,13-23,56.5,35.9,108.0,6,60.4,17.6,13.3,49.1,17-27,63.0,10-26,38.5,49.1
2,Tue Nov 14,240,South Alabama,"W, 102-46",H,67,152.0,1,72.7,14.9,44.0,54.5,22-32,68.8,12-23,52.2,41.8,68.5,1,27.5,17.9,29.6,35.0,9-31,29.0,5-29,17.2,48.3
3,Fri Nov 17,248,Mercer,"W, 98-67",H,74,132.7,1,66.1,14.9,33.3,49.1,27-40,67.5,8-19,42.1,32.2,90.7,5,43.0,12.2,22.7,35.9,23-50,46.0,3-14,21.4,21.9
4,Fri Nov 24,65,Ohio St.,"L, 92-81",N,72,112.0,8,54.5,16.6,21.9,46.4,17-26,65.4,9-30,30.0,53.6,127.2,22,64.0,19.4,29.2,60.0,17-32,53.1,10-18,55.6,36.0
5,Sat Nov 25,59,Oregon,"W, 99-91",N,78,127.4,1,57.0,20.6,48.5,43.8,23-40,57.5,9-24,37.5,37.5,117.1,20,57.2,16.7,31.4,21.7,29-49,59.2,7-20,35.0,29.0
6,Tue Nov 28,31,Clemson,"L, 85-77",H,70,109.6,6,42.5,10.0,32.6,38.8,12-32,37.5,11-35,31.4,52.2,121.0,18,62.5,15.7,24.2,25.0,21-39,53.8,11-21,52.4,35.0
7,Mon Dec 4,151,Arkansas St.,"W, 89-65",H,76,116.8,8,53.9,18.4,32.4,37.5,15-33,45.5,13-31,41.9,48.4,85.3,2,38.5,19.7,31.2,32.3,16-35,45.7,6-30,20.0,46.2
8,Sat Dec 9,2,Purdue,"L, 92-86",N,72,119.5,3,60.2,18.1,31.4,21.9,10-18,55.6,19-46,41.3,71.9,127.8,17,54.8,12.5,38.2,45.2,22-44,50.0,8-18,44.4,29.0
9,Sat Dec 16,16,Creighton,"L, 85-82",A,69,118.9,4,49.3,5.8,40.5,17.3,31-53,58.5,4-22,18.2,29.3,123.2,18,61.5,15.9,25.9,55.8,23-34,67.6,6-18,33.3,34.6


In [10]:
def run_simulation(n, team1, team2, browser, site):
    """
    Run simulations to predict outcomes of basketball games between two teams.
    The following uses Monte Carlo Simulation to predict the amount of possessions between two teams 
    and the amount of points per posession for both teams using random simulation of a normal distribution.

    Parameters:
    - n (int): Number of simulations to run.
    - team1 (str): Name of the first team.
    - team2 (str): Name of the second team.
    - browser: Web browser object used for web scraping.
    - site (str): Location of the game, must be one of 'HOME', 'AWAY', or 'NEUTRAL'.
    
    Returns:
    - results_df (DataFrame): DataFrame containing simulated game outcomes.
    - summary_df (DataFrame): DataFrame summarizing the number of wins for each team.
    """
    team1_df = get_gameplan(team1, browser)
    team2_df = get_gameplan(team2, browser)
    #team1 pace
    team1_pace = team1_df['pace'].tolist()
    team1_pace_std = np.std(team1_pace)
    team1_pace_mean = np.mean(team1_pace)
    # team1 offensive efficiecny 
    team1_off_eff = team1_df['off_eff']/100
    team1_off_eff_std = np.std(team1_off_eff)
    team1_off_eff_mean = np.mean(team1_off_eff)
    # team1 defensive efficiency
    team1_def_eff = team1_df['def_eff']/100
    team1_def_eff_std = np.std(team1_def_eff)
    team1_def_eff_mean = np.mean(team1_def_eff)
    # team2 pace
    team2_pace = team2_df['pace'].tolist()
    team2_pace_std = np.std(team2_pace)
    team2_pace_mean = np.mean(team2_pace)
    # team2 offensive efficiecny 
    team2_off_eff = team2_df['off_eff']/100
    team2_off_eff_std = np.std(team2_off_eff)
    team2_off_eff_mean = np.mean(team2_off_eff)
    # team2 defensive efficiency
    team2_def_eff = team2_df['def_eff']/100
    team2_def_eff_std = np.std(team2_def_eff)
    team2_def_eff_mean = np.mean(team2_def_eff)
    # get SOS
    temp = misc.get_pomeroy_ratings(browser)
    new = temp.dropna(axis=1)
    first_column = new.iloc[:, 0]
    final = pd.concat([temp['Team'], first_column], axis = 1)
    final['AdjEM'] = pd.to_numeric(final['AdjEM'], errors='coerce')
    final = final.dropna()
    # Error handling discovered in earlier stress testing for name Texas A&M
    if team1 == 'Texas A&M':
        team1 = 'Texas A'
    elif team2 == 'Texas A&M':
        team2 = 'Texas A'
    team1_sos = final[final['Team']==team1]['AdjEM'].values[0]
    team2_sos = final[final['Team']==team2]['AdjEM'].values[0]
    team1_sos_scored = team1_sos/100
    team2_sos_scored = team2_sos/100
    
    # get home court advantage
    hca = misc.get_hca(browser)
    hca['HCA'] = pd.to_numeric(hca['HCA'], errors='coerce')
    # Error handling for TAMU 
    if team1 == 'Texas A':
        team1 = 'Texas A&M'
    elif team2 == 'Texas A':
        team2 = 'Texas A&M'
    if site == 'HOME':
        hca = hca[hca['Team']==team1]['HCA'].values[0]
    elif site == 'AWAY':
        hca = hca[hca['Team']==team2]['HCA'].values[0]
    # Create results df
    results_df = pd.DataFrame(columns = [f'{team1}_Score', f'{team2}_score'])
    team1_wins = 0 
    team2_wins = 0
    ties = 0
    for i in range(0,n):
        # calculate pace of play
        team1_tempo = np.random.normal(team1_pace_mean, team1_pace_std)
        team2_tempo = np.random.normal(team2_pace_mean, team2_pace_std)
        min_tempo = min([team1_tempo, team2_tempo])
        max_tempo = max([team1_tempo, team2_tempo])
        # Heavier weight on slower pace of play as that has more gravity
        tempo = .6*min_tempo + .4*max_tempo
        # calculate team1 score
        team1_offense = np.random.normal(team1_off_eff_mean, team1_off_eff_std)
        team2_defense = np.random.normal(team2_def_eff_mean, team2_def_eff_std)
        team1_ppp = np.mean([team1_offense, team2_defense])
        # calculate team2 score
        team2_offense = np.random.normal(team2_off_eff_mean, team2_off_eff_std)
        team1_defense = np.random.normal(team1_def_eff_mean, team1_def_eff_std)
        team2_ppp = np.mean([team2_offense, team1_defense])
        # calculate score
        team1_score = team1_ppp * tempo
        team2_score = team2_ppp * tempo
        # adjust for SOS
        team1_score = team1_score + (team1_sos_scored*tempo)
        team2_score = team2_score + (team2_sos_scored*tempo)
        # adjust for site
        if site == 'HOME':
            team1_score = team1_score + (.5*hca)
            team2_score = team2_score - (.5*hca)
        elif site == 'AWAY':
            team1_score = team1_score - (.5*hca)
            team2_score = team2_score + (.5*hca)
        new_row = {f'{team1}_Score': team1_score, f'{team2}_score': team2_score}
        results_df = results_df.append(new_row, ignore_index = True)
        if team1_score > team2_score:
            team1_wins +=1
        elif team2_score > team1_score:
            team2_wins +=1

    summary_df = pd.DataFrame(columns = [f'{team1}_Wins', f'{team2}_Wins'])
    new_row_summary = {f'{team1}_Wins': team1_wins, f'{team2}_Wins': team2_wins}
    summary_df = summary_df.append(new_row_summary, ignore_index = True)
    return results_df, summary_df

In [11]:
simmed, summary = run_simulation(1000, 'Alabama', 'LSU', browser, 'AWAY')
simmed.describe()

https://kenpom.com/gameplan.php?team=Alabama
https://kenpom.com/gameplan.php?team=LSU


,Alabama_Score,LSU_score
count,1000.000000,1000.000000
mean,87.768515,82.844924
std,9.933961,8.565810
min,57.669980,52.061390
25%,80.750483,76.626704
50%,87.658921,82.447346
75%,94.531652,88.907205
max,120.319122,107.938075


In [12]:
summary

,Alabama_Wins,LSU_Wins
0,669,331
